In [1]:
import cv2
import numpy as np
import os
import tensorflow.keras as keras
from tensorflow.keras.models import model_from_json
import operator

In [2]:
json_file = open("model_bw.json", "r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)
loaded_model.load_weights("Save_file.h5")

In [ ]:
cap = cv2.VideoCapture(0)

categories = {0: 'Zero', 1: 'One', 2: 'Two', 3: 'Three', 4: 'Four', 5: 'Five'}




while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)   #mirror image
    
    # coordinates of the ROI
    x1 = int(0.6*frame.shape[1])
    y1 = 120
    x2 = frame.shape[1]-10
    y2 = int(0.5*frame.shape[1])
    #Drawing the ROI
    # The increment/decrement by 1 is to compensate for the bounding box
    cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255, 0, 0), 1)
    # Extracting the ROI
    roi = frame[y1:y2, x1:x2]
    roi = cv2.resize(roi, (68, 68))
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    _, test_image = cv2.threshold(roi, 120, 255, cv2.THRESH_BINARY)
    
    cv2.imshow("test", test_image)
    
#     Batch of 1
    result = loaded_model.predict(test_image.reshape(1,68,68,1))
    prediction = {'Zero': result[0][0],
                  'One': result[0][1],
                  'Two': result[0][2],
                  'Three': result[0][3],
                  'Four': result[0][4],
                  'Five': result[0][5]}
#     Sorting based on top prediction
    prediction = sorted(prediction.items(),key=operator.itemgetter(1), reverse=True)
    
#     Displaying the predictions
    cv2.putText(frame, prediction[0][0], (x1+100, y2+30), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,255), 1)
    cv2.imshow("Frame", frame)
    
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == 27: #esc key
        break

cap.release()
cv2.destroyAllWindows()